# Data Masters Case - Machine Learning Engineer: 
## Distributing GPU Training and Deploy (Cross Workspace) using TensorFlow 2 + Spark + MLFlow
### Marcos Vinícius Lisboa Melo - BigData & Analytics - vinicius.lisboa@f1rst.com.br

This notebook is used to perform requests from local environment to Deployed REST Api with the TensorFlow Model.

### Setting an environment to perform requests
F1rst we need to define the endpoints parameters and import all the necessary libs to perform a call:

In [67]:
import json
import requests
import os
import numpy as np
import pandas as pd
import cv2

Also the url, we need to create an environment variable to storage the token from API, it's not a best pratice hardcoded a token into a script. This variable was definied at Terminal and the name is `DATABRICKS_TOKEN_GCP`. To access `.env` file the bellow cell must to be executed. It's importante remember that the env var must to be definied locally with the Databricks generated token.

In [190]:
%reload_ext dotenv
%dotenv

In [194]:
url = 'https://357648146060830.0.gcp.databricks.com/model/mnist_cnn_data_masters/1/invocations'
headers = {'Authorization': f'Bearer {os.environ.get("DATABRICKS_TOKEN_GCP")}', 'Content-Type': 'application/json'}

### Creating Functions to compose data payload
After import image it's necessary redimensionate to 28x28 pixels format and convert to gray image with just one chanel, of course. So, we need to insert the converted and resized image into a numpy array named `sample`, because this is the standard format definied at TFX documentation to request. If we have more than one image, is just necessary append this images in array. After we create the array is necessary to convert to string format to compose the request JSON message. The key `instances` is also definied in documentation.

In [202]:
def create_formated_image(image, write_file=False, path=''):
    expected_width = 28
    expected_length = 28

    width = expected_width/image.shape[0]
    length = expected_length/image.shape[1]

    resize_image = cv2.resize(image, (0, 0), fx=width, fy=length)
    gray_image = cv2.cvtColor(resize_image, cv2.COLOR_BGR2GRAY)
    
    sample = [gray_image]
    sample = np.array(sample)

    sample_string = np.array2string(sample, separator=',').replace('\n','')

    json_string = '{"instances":' + sample_string + '}'
    data_json = json.loads(json_string)
    
    if write_file == True:
        json_object = json.dumps(data_json, indent=4)
        with open(path, 'w') as outfile:
            outfile.write(json_object)
    
    return data_json

Almost at least, we need a `score_model` function to compose the request and receive response.

In [216]:
def score_model(image, url=url, headers=headers):
    
    data_json = json.dumps(create_formated_image(image))
    
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    
    prediction = response.json()['predictions']
    prediction = np.argmax(prediction)
    
    return prediction

### Perform a predict request
Calling `score_model` function and passing the loaded image and the default url and headers before setted:

In [212]:
image = cv2.imread('/Users/t780496/Documents/notebooks_python/data_masters/mnist_test_images/mnist_teste_2.jpeg')

In [217]:
number = score_model(image, url, headers)
print('Current prediction number is: '+ str(number))

Current prediction number is: 3
